In [1]:
import os
import pathlib
import site

In [2]:
import logging
import warnings

from anomalib.utils.loggers import configure_logger, get_experiment_logger


logger = logging.getLogger("anomalib")
configure_logger(level="ERROR") # "<DEBUG, INFO, WARNING, ERROR>"

To use wandb logger install it using `pip install wandb`


In [3]:
from pytorch_lightning import Trainer, seed_everything

from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.data.utils import TestSplitMode
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks

## Monkey Patch

In [4]:
from MyFeatureExtractor import FeatureExtractor
import anomalib.models.padim.torch_model as ptm
ptm.FeatureExtractor = FeatureExtractor

## Config

In [5]:
package_path = site.getsitepackages()[0]
for p in site.getsitepackages():
    if "site-package" in p:
        package_path = p
        break

package_path

'C:\\Users\\takanari\\miniconda3\\envs\\python39_dev_anomalib\\lib\\site-packages'

In [6]:
"""
SET MODEL NAME
"""
model_name = "padim"
config_path = os.path.join(package_path, f"anomalib/models/{model_name}/config.yaml")
config = get_configurable_parameters(model_name=model_name, config_path=config_path)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\anomalib\config\config.py:280: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(


In [7]:
"""
DATASET SETTING 
"""
config.dataset.train_batch_size = 4
config.dataset.eval_batch_size = 4

dict(config.dataset)

{'name': 'mvtec',
 'format': 'mvtec',
 'path': './datasets/MVTec',
 'category': 'bottle',
 'task': 'segmentation',
 'train_batch_size': 4,
 'eval_batch_size': 4,
 'num_workers': 8,
 'image_size': [256, 256],
 'center_crop': None,
 'normalization': 'imagenet',
 'transform_config': {'train': None, 'eval': None},
 'test_split_mode': 'from_dir',
 'test_split_ratio': 0.2,
 'val_split_mode': 'same_as_test',
 'val_split_ratio': 0.5,
 'tiling': {'apply': False, 'tile_size': None, 'stride': None, 'remove_border_count': 0, 'use_random_tiling': False, 'random_tile_count': 16}}

In [8]:
dict(config.logging)

{'logger': [], 'log_graph': False}

In [9]:
dict(config.metrics)

{'image': ['F1Score', 'AUROC'],
 'pixel': ['F1Score', 'AUROC'],
 'threshold': {'method': 'adaptive', 'manual_image': None, 'manual_pixel': None}}

In [10]:
"""
MODEL SETTING

REF: https://github.com/JohnnyHopp/PaDiM-EfficientNetV2/blob/master/main.py
"""
# config.model.backbone = "resnet18"
# config.model.layers = ['layer1.-1', 'layer2.-1', 'layer3.-1']
# config.model.n_features = 100

# config.model.backbone = "wide_resnet50_2"
# config.model.layers = ['layer1.-1', 'layer2.-1', 'layer3.-1']
# config.model.n_features = 550

# config.model.backbone = "tf_efficientnet_b5_ns"
# config.model.layers = ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 100

# config.model.backbone = "tf_efficientnet_b7_ns"
# config.model.layers = ['blocks.1.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 100

# config.model.backbone = "tf_efficientnetv2_m_in21k"
# config.model.layers = ['blocks.2.-1', 'blocks.3.-1', 'blocks.4.-1']
# config.model.n_features = 400

config.model.backbone = "convnext_base_in22k"
config.model.layers = ['stages_0.blocks.-1', 'stages_1.blocks.-1', 'stages_2.blocks.-1']
config.model.n_features = 300

dict(config.model)

{'name': 'padim',
 'backbone': 'convnext_base_in22k',
 'pre_trained': True,
 'layers': ['stages_0.blocks.-1', 'stages_1.blocks.-1', 'stages_2.blocks.-1'],
 'normalization_method': 'min_max',
 'input_size': [256, 256],
 'n_features': 300}

In [11]:
dict(config.optimization)

{'export_mode': None}

In [12]:
def transform_path(path_string):
    p = pathlib.Path(path_string)
    
    # もし最後がディレクトリでない場合、その親ディレクトリを取得
    if not path_string.endswith(('/', '\\')):
        p = p.parent
    
    # パスの部分をリストとして取得
    parts = list(p.parts)

    # 新しいパス形式を組み立て
    new_path = f"{parts[0]}/{parts[3]}_{parts[1]}_"
    return new_path

In [13]:
config.project.path = transform_path(config.project.path) + f"{config.model.backbone}_n{config.model.n_features}"

dict(config.project)

{'seed': 42,
 'path': 'results/bottle_padim_convnext_base_in22k_n300',
 'unique_dir': False}

In [14]:
config.trainer.default_root_dir = transform_path(config.trainer.default_root_dir) + f"{config.model.backbone}_n{config.model.n_features}"

dict(config.trainer)

{'enable_checkpointing': True,
 'default_root_dir': 'results/bottle_padim_convnext_base_in22k_n300',
 'gradient_clip_val': 0,
 'gradient_clip_algorithm': 'norm',
 'num_nodes': 1,
 'devices': 1,
 'enable_progress_bar': True,
 'overfit_batches': 0.0,
 'track_grad_norm': -1,
 'check_val_every_n_epoch': 1,
 'fast_dev_run': False,
 'accumulate_grad_batches': 1,
 'max_epochs': 1,
 'min_epochs': None,
 'max_steps': -1,
 'min_steps': None,
 'max_time': None,
 'limit_train_batches': 1.0,
 'limit_val_batches': 1.0,
 'limit_test_batches': 1.0,
 'limit_predict_batches': 1.0,
 'val_check_interval': 1.0,
 'log_every_n_steps': 50,
 'accelerator': 'auto',
 'strategy': None,
 'sync_batchnorm': False,
 'precision': 32,
 'enable_model_summary': True,
 'num_sanity_val_steps': 0,
 'profiler': None,
 'benchmark': False,
 'deterministic': False,
 'reload_dataloaders_every_n_epochs': 0,
 'auto_lr_find': False,
 'replace_sampler_ddp': True,
 'detect_anomaly': False,
 'auto_scale_batch_size': False,
 'plugins':

In [15]:
dict(config.visualization)

{'show_images': False,
 'save_images': True,
 'log_images': True,
 'image_save_path': None,
 'mode': 'full'}

## Train

In [16]:
datamodule = get_datamodule(config)
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\anomalib\utils\callbacks\__init__.py:142: UserWarning: Export option: None not found. Defaulting to no model export
  warnings.warn(f"Export option: {config.optimization.export_mode} not found. Defaulting to no model export")


In [17]:
model.model

PadimModel(
  (feature_extractor): FeatureExtractor(
    (feature_extractor): FeatureListNet(
      (stem_0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (stem_1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
      (stages_0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU()
              (drop1): Dropout(p=0.0, inplace=False)
              (fc2): Linear(in_features=512, out_features=128, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=

In [18]:
trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
logger.info("Training the model.")
trainer.fit(model=model, datamodule=datamodule)

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\pytorch_lightning\core\optimizer.py:183: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

C:\Users\takanari\miniconda3\envs\python39_dev_anomalib\lib\site-packages\pytorch_lightning\loops\optimization\optimizer_loop.py:138: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")


Validation: 0it [00:00, ?it/s]

# Test

In [19]:
weight_file_path = trainer.checkpoint_callback.best_model_path
weight_file_path

'C:\\Users\\takanari\\Documents\\work\\dev_anomalib_with_docker\\work\\results\\bottle_padim_convnext_base_in22k_n300\\weights\\lightning\\model.ckpt'

In [20]:
logger.info("Loading the best model weights.")
load_model_callback = LoadModelCallback(weights_path=weight_file_path)
trainer.callbacks.insert(0, load_model_callback)

_=trainer.test(model=model, datamodule=datamodule)


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       image_AUROC                  1.0
      image_F1Score                 1.0
       pixel_AUROC          0.9777892827987671
      pixel_F1Score         0.6716136932373047
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [21]:
model.model.feature_extractor.out_dims

[128, 256, 512]

## Appendix

In [22]:
import timm

In [23]:
# timm.create_model(
#     "efficientnet_b5",
#     pretrained=True,
#     features_only=True,
#     exportable=True,
# )

In [24]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224',
 'coatnet_1_rw_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_rw_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_atto',
 'convnext_atto_